# COGS 118A - Project Checkpoint

# Names

Hopefully your team is at least this good. Obviously you should replace these with your names.

- Sidney Ma
- Sean Rote
- Sarah Horne
- Kevin Su
- Jack Zhou

# Abstract 
In this age, music has become an important part of people’s lives and an incredibly important part of their individuality, so why are our music recommendation systems based on the data from other users? Our research focuses on Spotify, a popular Swedish music service. We believe a better algorithm would focus on the content and composition of the songs to recommend objectively similar songs to users. Our research aims to look at a simpler classification, the first step towards creating this type of classification algorithm is a supervised machine learning algorithm that classifies songs into different genres based on different features of the song. The dataset that we have looks at different features of the songs like danceability, loudness, and tempo and also provides the true label for the genre of the song. The success of our algorithm will be based on how accurately the algorithm can predict the genre of a song, based on these features.


# Background

A plethora of research has gone into recommendation algorithms, picking apart every pro and con of the incredibly common mechanism. Research into these algorithms doesn’t converge to provide one technique as the most effective or efficient. Our research plans to specifically place emphasis on Spotify, the incredibly popular Swedish music service. Spotify’s current algorithm pulls from other users data to compare what users with similar music preferences listen to. This recommendation system is called “collaborative filtering”. “If user A has enjoyed songs X, Y and Z, and user B has enjoyed songs X and Y (but haven't heard Z yet), we should recommend song Z to them” <a name="mustomorrow"></a>[<sup>[1]</sup>](#mustomorrownote).

Instead, if we could create a new algorithm that focused on the content of songs, we may be able to use machine learning to create a cheaper, better system. This system would focus on quantifying different aspects of songs and finding similar songs to recommend to users. For this theoretical recommendation algorithm, one of the most important factors in determining a similar song, is identifying songs in the same genre. But how is genre classified? According to Merriam-Webster, a genre is “a category of artistic, musical, or literary composition characterized by a particular style, form, or content” <a name="merriam"></a>[<sup>[2]</sup>](#merriamnote). The definition of different genres is subjective. When a singer or songwriter creates a new song, they are responsible for categorizing that song into genres, yet no standards exist for such classification. 

Therefore, in order to produce a machine learning algorithm for music recommendation, one must first be able to classify songs into standardized genres based on the objective content of the song; Our algorithm aims to do just that. Utilizing features of music like danceability, energy, and instrumentalness, we will classify different songs into popular genres.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds. Use a minimum of 2 or 3 citations, but we prefer more. You need enough citations to fully explain and back up important facts. 

# Problem Statement

As we stated above, the current song recommendation doesn’t objectively rely on the similarity of the songs but similar users’ listening patterns. The purpose of our project is to focus on the very first step of this more efficient and intricate song recommendation system:  identifying the genre of the current song the user is listening to, hence creating a better recommendation algorithm for spotify recommended songs. We will focus on more technical features of the song itself instead of a similar user’s listening pattern. Our objective is to find the best regression algorithm which generates the most accurate prediction of the song’s genre. 

# Data

There is no shortage of Spotify data that can be found online. We will be working with this dataset found on Kaggle.com:
Spotify Tracks Dataset
- Link: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset
- 114,000 rows, 20 columns
- Each row includes objective information about the song (tempo, length, etc.), subjective information (danceability, energy, etc.), genre, and more.
- Includes over 125 genres.

After obtaining the dataset, the next step is to clean it. We dropped the first columns that would be irrelevant to our analysis, like the track ID, the artist name and the album name. We decided that some features will likely not be as relevant. Because of this, we decided to drop explicit, popularity, and mode. We also decided to drop key, because some of the songs have keys that could not be detected, and there may be a pattern to which song’s keys could not be detected. This meant we could not simply drop the songs where no key was detected.

When trying to fit our classifier, we discovered that our data set is entirely too large. Our data set has 114 genres of 1,000 songs each. We decided that we should do less genres, so we decided on a list of 8 genres to classify. We chose to keep pop, country, edm, rock, classical, hip-hop, jazz, and gospel.

## EDA

In [1]:
# Import modules and functions
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.metrics import zero_one_loss, accuracy_score, roc_auc_score, f1_score,auc
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

import matplotlib.pyplot as plt

In [2]:
# Import dataset
df = pd.read_csv('dataset.csv')
df.shape

(114000, 21)

The original dataset has over a million data points, so we'll reduce it.

In [3]:
# Only include rows of the desired genres
df = df.loc[df['track_genre'].isin(
    ['pop','country','edm','rock','classical','hip-hop','jazz','gospel']
)]
df['track_genre'].unique()

array(['classical', 'country', 'edm', 'gospel', 'hip-hop', 'jazz', 'pop',
       'rock'], dtype=object)

We'll keep track of the true labels to compare against later.

In [4]:
True_labels = df['track_genre']

As mentioned above, we decided that some features would not be relevant, so we'll remove them from the dataset.

In [5]:
# Remove irrelevant columns
df = df.drop(
    ['Unnamed: 0','track_id','artists',
     'album_name','explicit','popularity',
     'mode','key','track_name'], axis = 1)
df

,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
16000,298266,0.643,0.268,-15.073,0.0900,0.5930,0.000002,0.316,0.620,143.813,4,classical
16001,482586,0.484,0.898,-4.132,0.1640,0.3650,0.000000,0.091,0.680,91.975,4,classical
16002,219437,0.608,0.638,-6.008,0.0292,0.5810,0.017200,0.448,0.439,140.109,4,classical
16003,299146,0.695,0.293,-16.278,0.0431,0.5960,0.015800,0.132,0.637,143.804,4,classical
16004,387716,0.583,0.308,-18.303,0.0465,0.5810,0.010600,0.257,0.241,118.226,4,classical
...,...,...,...,...,...,...,...,...,...,...,...,...
91995,253626,0.440,0.838,-6.196,0.0423,0.0956,0.000000,0.473,0.619,138.908,4,rock
91996,253626,0.440,0.838,-6.196,0.0423,0.0956,0.000000,0.473,0.619,138.908,4,rock
91997,253626,0.440,0.838,-6.196,0.0423,0.0956,0.000000,0.473,0.619,138.908,4,rock
91998,253626,0.440,0.838,-6.196,0.0423,0.0956,0.000000,0.473,0.619,138.908,4,rock


In [6]:
# How many songs per genre?
True_labels.value_counts()

classical    1000
country      1000
edm          1000
gospel       1000
hip-hop      1000
jazz         1000
pop          1000
rock         1000
Name: track_genre, dtype: int64

This dataset seems usable. There are 1000 songs per genre, and each column specifies a feature that could be useful for identifying the genre.

# Proposed Solution

We will utilize the datasets of spotify on Kaggle that were extracted through Spotify API and we will determine a fix set of features for anaylyzing and representing a song. We are going to use a .33 test size for test, train split. The main regression algorithm we are going to use for is onevsrest classfier. First, we will decide what is the best set of features to determine the similarity of the songs. The datasets provided us various types of features such as dancibility, loudness, key, and livenss, etc. As some features may not be as representative as other features, it’s important to select the most indicative features to feed into our algorithm. We also will need to select the most optimal hyperparameter for the onevsrest classifier algorithm. As we running the algorithm, we want to make sure that both generalization error and testing error will be minimized. We will use a benchmark algorithm to generate truth label and compare with our testing label. 

# Evaluation Metrics

The evaluation metric that helps quantify the performance of our model is going to be a multiclass confusion matrix. A multiclass confusion matrix is a table that summarizes the performance of a multiclass classification model. In the case of our multiclass confusion matrix we will be comparing the different genres where we will be looking for diagonal results. The diagonal of the matrix represents the number of correctly classified samples for each class. And the off-diagonal elements represent the misclassified samples. We can then take values from our confusion matrix to calculate precision, recall, accuracy, and F1 score. 


# Preliminary results

After cleaning the large data set we were able to narrow our true labels down from 110 to 8 genres as many were sub-genres. We then compared three genres to each other, first being country and rock, which we thought were the most similar, and then comparing classical music to rock, which we thought were very different. 

Country vs Rock          |  Rock vs Classical
:-----------------------:|:-----------------------:
<img src="images/image3.png" width="400"/>  |  <img src="images/image4.png" width="400"/>

We are able to see both the similarity between country and rock due to the data spread and the difference between rock and classical. Moving forward to our model we decided to do different confusion matrices that had different test sizes one being 0.20 and the other being 0.33. This demonstrated that increasing the test size helped our model classify genres better.

test_size=0.20 | test_size=0.33
:-----------------------:|:-----------------------:
<img src="images/image2.png" width="400"/>  |  <img src="images/image1.png" width="400"/>


# Ethics & Privacy

There are a variety of ethical issues to consider, including privacy concerns, managing dataset bias, avoiding unreasonable conclusions, and setting realistic expectations as to what our tool will actually be capable of.
Privacy concerns will probably be the easiest to manage. In order to avoid privacy breaches, it is important for us to make sure that the data we work with does not cross into personal user information. As of now, the data we are using is safe.

Dataset bias is a significant issue to consider. Because we are planning to make a tool to be used by all music listeners, we would ideally want it to be equally useful for everyone. However, since our model will be trained on certain genres of music more than others, it will obviously be much better at identifying only those specific genres. This is an especially important issue to consider in terms of equality – since different demographics of people tend to listen to different genres, it is important to make sure that our tool does not favor one demographic over another. We will mention this issue in our limitations, but to avoid it as much as possible, our best and most simple option is to make sure we have a fair number of songs in each genre.

It is also important that the findings we come across in our project do not lead to erroneous conclusions. For instance, if we find that our tool is much better at identifying hip hop than it is at identifying EDM, we would not take this to mean that hip hop is simply “less diverse” than EDM – the result might just have to do with the limited dataset our model has been trained on. It is not only important that we don’t justify unreasonable conclusions in our paper, but also important to be clear enough that readers would not come to these conclusions either.

We should make it clear in our paper what our tool should be expected to do, and be very clear about its many limitations. For instance, our tool will obviously not work on every single song – there are probably many songs that our tool will not be able to categorize. At the same time, we should make sure that our tool is still able to provide some utility to the user.


# Team Expectations 

- Team members will meet at least once a week
- Team members should be responsive in the team group chat
- Aim to finish assignments at least 24 hours before the deadline
- If a team member misses a meeting we will set up a set time to fill them in
- If conflict should arise we should resolve through group communication
- Workload expectation for the group is assumed equal load among each other


# Project Timeline Proposal

| Date & Time | Agenda | Goal |
|:------------|:-------|:-----|
| 2/21 14:00  | Peer reviews of proposal | Finish the proposal and ready for turning in |
| 2/27 16:00  | Assign jobs to each team member | Have dataset cleaning done and plan for out exploratory data analysis|
| 3/5 10:00 | Meet at Geisel to work on EDA and checkpoint | Get confusion matrix; plan on finishing checkpoint  |
| 3/6 16:00 | Discuss EDA progress and brainstorm for any roadblock | Continue to prepare our analysis and make the observation understandable by using metrics; finish checkpoint
| 3/13 15:00 | Discuss the progress of the entire project, see what is still missing | Complete the analysis and begin to draft our conclusion and report
| 3/20 11:00 | Each team member demonstrate their part, and complete the group report | Finish up the report

# Footnotes
<a name="mustomorrownote"></a>1.[^](#mustomorrow): Pastukhov, D. (9 Feb 2022). Inside Spotify’s Recommender System: A Complete Guide to Spotify Recommendation Algorithms. *Music Tomorrow*. https://www.music-tomorrow.com/blog/how-spotify-recommendation-system-works-a-complete-guide-2022.<br> 
<a name="merriamnote"></a>2.[^](#merriam): Genre Definition. *Merriam-Webster*. https://www.merriam-webster.com/dictionary/genre.<br>
